## How to create a Deep Ensemble for MNIST

In [ ]:
import tensorflow as tf

try:
  import uncertainty_wizard as uwiz
except ModuleNotFoundError as e:
  # Uncertainty wizard was not installed. Install it now (we're probably on colab)
  !pip install uncertainty_wizard
  import uncertainty_wizard as uwiz

**Step 1: Downloading Preprocessing the data**

This is the same that we would do on any regular keras mnist classifier,
except that we do not have to one-hot encode the test labels, as uncertainty wizards quantifiers
will determine the winning class for us

In [ ]:
# Lets cache the train data on the file system,
# and at the same time also prepare the test data for later
_,(x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_test = (x_test.astype('float32') / 255).reshape(x_test.shape[0], 28, 28, 1)


**Step 2: Define the model creation & training process in a picklable function**

Just create a function at the root of your file, using plain tensorflow code. 
The function should return a newly created model and a second return value (typically the training history).

This function will be called repetitively to create the atomic models in the ensemble.
The optional return values will be collected and returned after the creation of the ensemble.

In [ ]:
def model_creation_and_training(model_id: int):
    import tensorflow as tf

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dense(10, activation='softmax'))
    model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
    (x_train, y_train), (_,_) = tf.keras.datasets.mnist.load_data()
    x_train = (x_train.astype('float32') / 255).reshape(x_train.shape[0], 28, 28, 1)
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
    # Note that we set the number of epochs to just 1, to be able to run this notebook quickly
    # Set the number of epochs higher if you want to optimally train the network
    fit_history = model.fit(x_train, y_train, validation_split=0.1, batch_size=32, epochs=1,
                      verbose=1, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)])
    return model, fit_history.history


**Step 3: Create Ensemble**

Let's create a Lazy Ensemble instance, i.e., a definition of how many atomic models should be included in our ensemble,
where they should be persisted, ... Note that this first call does not create or train any models and is thus super fast.

After this definition, we can create the atomic models in the lazy ensemble using the function defined above.

In [ ]:
ensemble = uwiz.models.LazyEnsemble(num_models=2, # For the sake of this example. Use more in practice!
                                    model_save_path="/tmp/ensemble",
                                    # Colab infrastructure is relatively weak.
                                    # Thus, lets disable multiprocessing and train on the main process.
                                    # Any argument >= 1 would result in (typically more efficient) multiprocessing
                                    # on a more powerful machine
                                    default_num_processes=0)
# Creates, trains and persists atomic models using our function defined above
training_histories = ensemble.create(create_function=model_creation_and_training)

**Step 4: Make predictions and get the uncertainties and confidences**

If your test data is a numpy array, its as easy as shown in the code below.

For customized prediction procedures, 
or a non-numpy test set, check out the documentation for ensemble.quantify_predictions where you
can hook up an arbitrary prediction function - similar to the training function defined and used in step 2 and 3

In [ ]:
# Get two one-dimensional np arrays: One containing the predictions and one containing the confidences
predictions, confidences = ensemble.predict_quantified(x_test,
                                                       quantifier='mean_softmax')
